In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

## Считывание данных

In [ ]:
train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

train.sort_values('sig_id', inplace=True)
train_scored.sort_values('sig_id', inplace=True)

test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

## Проверка на nans

In [ ]:
sum(train.isna().sum())

Отлично, все значения есть

## Признаки

In [ ]:
train.head()

`sig_id` - айдишник, `cp_type`, `cp_time`, `cp_dose` - категориальные переменные. Посмотрим на остальные признаки, видим, что некоторые из них начинаются на `g-`, некоторые на `c-`. `g` - гены, `c` - клетки. Посмотрим, покрывают ли они все признаки. Всего в модели 875 признаков

In [ ]:
cat_features = ['cp_type', 'cp_time', 'cp_dose']
gen_features = [column for column in train.columns if column.startswith('g-')]
cell_features = [column for column in train.columns if column.startswith('c-')]

In [ ]:
print('Число категориальных признаков: ', len(cat_features))
print('Число генных признаков: ', len(gen_features))
print('Число клеточных признаков: ', len(cell_features))

Как видим, это все признаки

### Категориальные признаки

In [ ]:
train.set_index('sig_id', inplace=True)
train_scored.set_index('sig_id', inplace=True)
test.set_index('sig_id', inplace=True)

In [ ]:
for feature in cat_features:
    train[feature] = train[feature].astype('category')
    test[feature] = test[feature].astype('category')

In [ ]:
index = list(train['cp_type'].value_counts().index)
height = list(train['cp_type'].value_counts())

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

for i, feature in enumerate(cat_features, 1):
    plt.subplot(1, 3, i)
    plt.title(feature)
    index = list(train[feature].astype('str').value_counts().index)
    height = list(train[feature].value_counts())
    plt.bar(x=index, height=height, width=0.5)
plt.show()

Видно, что довольно мало `ctl_vehicle` значения для признака `cp_type`, остальные признаки распределены равномерно.

#### Связь с таргетом

Попробуем оценить зависимость таргета от категориальных переменных. Для начала посмотрим на процент единиц для каждой из целевой переменной

In [ ]:
plt.figure(figsize=(10, 7))
plt.hist(train_scored.mean(axis=0) * 100, bins=20)
plt.xlabel('Percent of 1', fontsize=15)
plt.show()

Как видим, все целевые переменные крайне несбалансированны. Посмотрим зависимость каждой целевой переменной от категориальных признаков, а именно посмотрим, есть ли целевые переменные, которые принимают константное значение в зависимости от значения категориальной переменной

In [ ]:
const_target = {}

for feature in cat_features:
    const_target[feature] = {}
    for value in train[feature].unique():
        sum_target = train_scored.loc[train[train[feature] == value].index].sum(axis=0)
        const_target[feature][value] = -1 * np.ones_like(sum_target)
        const_target[feature][value][sum_target == 0] = 0

In [ ]:
for feature in const_target:
    print(feature)
    for value in const_target[feature]:
        print(value)
        print('Const zeros target: ', sum(const_target[feature][value] == 0))
    print()

Поразительное наблюдение: при `cp_type` = `crt_vehicle` все целевые переменные равны нулю, попробуем выкинуть их из рассмотрения в принципе, а ответ на них считать равным 0.

In [ ]:
print(np.where(const_target['cp_time'][24] == 0)[0])
print(np.where(const_target['cp_time'][72] == 0)[0])
print(np.where(const_target['cp_dose']['D2'] == 0)[0])

In [ ]:
print(sum(train_scored[train_scored.columns[34]]))
print(sum(train_scored[train_scored.columns[82]]))

В остальных случаях, когда константный ноль в признаке по значению категориальной переменной -- это два признака со всего одним значеним. В принципе, на предсказании их тоже можно заполноять константой нулем.

### Вещественные признаки

In [ ]:
train[np.random.choice(gen_features, 9, replace=False)].hist(figsize=(15, 15), bins=20)
plt.show()

In [ ]:
train[np.random.choice(cell_features, 9, replace=False)].hist(figsize=(15, 15), bins=20)
plt.show()

Распределения довольно одинаковы как для генов, так и для клеток. Возможно, полезная информация содержится в хвостах

In [ ]:
plt.figure(figsize=(12, 8))
train[gen_features].std().hist(density=True, bins=20)
train[cell_features].std().hist(density=True, bins=20)
plt.title('Распределение std по признакам на трейне', fontsize=15)
plt.legend(['Генные признаки', 'клеточные признаки'], fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
test[gen_features].std().hist(density=True, bins=20)
test[cell_features].std().hist(density=True, bins=20)
plt.title('Распределение std по признакам на тесте', fontsize=15)
plt.legend(['Генные признаки', 'клеточные признаки'], fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
train[gen_features].mean().hist(density=True, bins=20)
train[cell_features].mean().hist(density=True, bins=20)
plt.title('Распределение mean по признакам на трейне', fontsize=15)
plt.legend(['Генные признаки', 'клеточные признаки'], fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
test[gen_features].mean().hist(density=True, bins=20)
test[cell_features].mean().hist(density=True, bins=20)
plt.title('Распределение mean по признакам на тесте', fontsize=15)
plt.legend(['Генные признаки', 'клеточные признаки'], fontsize=15)
plt.show()

Раз нам скорее всгео важны хвосты, попробуем отбросить признаки с совсем маленьким среднеквадратичным отклонением. Судя по графикам выше, это относится к генным признакам, с отклонением 0.8, возмем порог по дисперсии - 0.7

In [ ]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

var_threshold = VarianceThreshold(threshold=0.7)

real_data = train.append(test)
real_data_transformed = var_threshold.fit_transform(real_data.iloc[:, 4:])

train_real_transformed = real_data_transformed[ :train.shape[0]]
test_real_transformed = real_data_transformed[train.shape[0]: ]


train = pd.DataFrame(train[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),
                     columns=['sig_id','cp_type','cp_time','cp_dose'])
train = pd.concat([train, pd.DataFrame(train_real_transformed)], axis=1)


test = pd.DataFrame(test[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                    columns=['sig_id','cp_type','cp_time','cp_dose'])
test = pd.concat([test, pd.DataFrame(test_real_transformed)], axis=1)

Можно грамотнее обработать вещественные признаки. Пока обойдемся этим, остальное выделение важного оставим нейросети.

## Дополнительная предобработка

Уберем данные, где `cp_type` = `ctl_vehicle`. У признака останется одно значение, соответственно весь признак можно убрать

In [ ]:
train.set_index('sig_id', inplace=True)
need_indexes = train[(train['cp_type'] != 'ctl_vehicle')].index

In [ ]:
# в предсказании заполним нулями
train = train.loc[need_indexes].copy()
train_scored = train_scored.loc[need_indexes].copy()

test = test[test['cp_type'] != 'ctl_vehicle'].copy()

train.reset_index(inplace=True)
train_scored.reset_index(inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
train.drop('cp_type', axis=1, inplace=True)
test.drop('cp_type', axis=1, inplace=True)

In [ ]:
cp_time_item2index = {elem: index for index, elem in enumerate(train['cp_time'].unique(), 0)}
cp_dose_item2index = {elem: index for index, elem in enumerate(train['cp_dose'].unique(), 0)}

In [ ]:
train['cp_time'] = train['cp_time'].map(cp_time_item2index)
train['cp_dose'] = train['cp_dose'].map(cp_dose_item2index)

test['cp_time'] = test['cp_time'].map(cp_time_item2index)
test['cp_dose'] = test['cp_dose'].map(cp_dose_item2index)

In [ ]:
train.head()

`cp_time` оставим как есть, будем считать ординальной переменной

## Датасет

In [ ]:
class Data(Dataset):
    def __init__(self, data: pd.DataFrame, target: pd.DataFrame):
        self.data = data[data.columns[1:]]
        self.y = target[target.columns[1:]]
        
    def __getitem__(self, index):
        return torch.tensor(self.data.iloc[index], dtype=torch.float), torch.tensor(self.y.iloc[index], dtype=torch.float)
    
    def __len__(self):
        return len(self.data)
    
    
class TestData(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data[data.columns[1:]]
        
    def __getitem__(self, index):
        return torch.tensor(self.data.iloc[index], dtype=torch.float)
    
    def __len__(self):
        return len(self.data)

In [ ]:
train_indexes, test_indexes = train_test_split(np.arange(len(train)), train_size=0.8, random_state=42)

In [ ]:
train_data = Data(train.loc[train_indexes], train_scored.loc[train_indexes])
valid_data = Data(train.loc[test_indexes], train_scored.loc[test_indexes])

## Обучение

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=1)
valid_loader = DataLoader(valid_data, batch_size=16, num_workers=1)

In [ ]:
class DummyModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_classes):
        super().__init__()
        self.linear_in = nn.Linear(num_features, hidden_size)
        self.norm = nn.BatchNorm1d(hidden_size)
        self.drop = nn.Dropout(0.2)
        self.linear_out = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.linear_in(x)
        x = self.norm(x)
        x = F.relu(self.drop(x))
        x = self.linear_out(x)
        return x

In [ ]:
model = DummyModel(train.shape[1] - 1, 512, train_scored.shape[1] - 1)
model.to(device)

In [ ]:
loss_func = nn.BCEWithLogitsLoss(reduction='none')
loss_func.to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, gamma=0.95, step_size=1)
#scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=1e-2, total_steps=30)

In [ ]:
num_epochs = 10

for i in range(num_epochs):
    epoch_loss = torch.zeros(train_scored.shape[1] - 1)
    for batch_idx, batch in enumerate(train_loader, 1):
        output = model(batch[0].to(device))
        target = batch[1].to(device)
        loss = loss_func(output, target)
        loss.mean().backward()

        optimizer.step()
        optimizer.zero_grad()
    #scheduler.step()
    
    model.eval()
    print(f"EPOCH {i+1}")
    train_loss = torch.zeros(train_scored.shape[1] - 1)
    with torch.no_grad():
        for batch in train_loader:
            output = model(batch[0].to(device))
            target = batch[1].to(device)
            loss = loss_func(output, target)
            batch_loss = loss.cpu().data.mean(dim=0)
            train_loss += batch_loss
        print("Train loss", float((train_loss / len(train_loader)).mean()))
    
    log_loss = torch.zeros(train_scored.shape[1] - 1)
    test_loss = 0
    with torch.no_grad():
        for batch in valid_loader:
            output = model(batch[0].to(device))
            target = batch[1].to(device)
            loss = loss_func(output, target)
            
            batch_loss = loss.cpu().data.mean(dim=0)
            test_loss += float(loss.cpu().data.mean())
            log_loss += batch_loss
        print("Log loss", float((log_loss / len(valid_loader)).mean()))
        print("Test loss", test_loss / len(valid_loader))
            
    
    model.train()

## Predict

In [ ]:
sub_ex = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
test_data = TestData(test)
test_loader = DataLoader(test_data, batch_size=16, num_workers=1)

In [ ]:
model.eval()
preds = []
with torch.no_grad():
    for batch in test_loader:
        preds.append(model(batch))
        
preds = torch.cat(preds, dim=0)
preds = torch.sigmoid(preds)

In [ ]:
res = pd.DataFrame(test['sig_id'])

In [ ]:
res = pd.concat((res, pd.DataFrame(preds.cpu().numpy())), axis=1)

In [ ]:
res = sub_ex[['sig_id']].merge(res, on='sig_id', how='left')
res = res.fillna(1e-5)
res.columns = sub_ex.columns

In [ ]:
res.to_csv('/kaggle/working/submission.csv', index=False)

## TODO

1. Уменьшение переобучения
2. CV для более грамотной валидации
3. Возможно попробовать CNN. Может соседство признаков в выборке имеет смысл
4. PCA вещественных признаков как новые признаки
5. Получше посмотреть на выбросы по вещественным признакам, возможно они определяют единички в таргете
6. Что-то для учета имбаланса. Вес классов? Focal Loss?
7. Подборка гиперпараметров сети и оптимизатора
8. Попробовать бустинг
9. Поумнее использовать категориальный признак времени